In [1]:
# Importação de bibliotecas

import pandas as pd # importar pandas para leitura, manipulação e agregação de dados tabulares
import numpy as np # importar numpy para operações numéricas e suporte matemático
from sklearn.preprocessing import StandardScaler # Importar o StandardScaler para padronizar variáveis numéricas
from sklearn.cluster import KMeans # importar KMeans para agrupar clientes com base em similiaridade


In [2]:
# Carregamento do Dataset

df = pd.read_excel("../data/online_retail_II.xlsx", sheet_name="Year 2010-2011") # Lê o arquivo Excel e define sheet_name a aba utilizada

df_sales = df[(df["Quantity"] > 0) & (df["Price"] > 0)].copy() # Filtrar registro eliminando devoluções e preços inválidos, criando uma cópia fisica
df_sales["Revenue"] = df_sales["Quantity"] * df_sales["Price"]

df_sales = df_sales[df_sales["Customer ID"].notna()] # Remove registro sem identificação de cliente para calcular RFM que é por cliente

In [8]:
# Cosntrução do RFM

snapshot_date = df_sales["InvoiceDate"].max() + pd.Timedelta(days=1) # Define a daa de referência usando última data de compra + 1 dia para evitar recency = 0

rfm = (
    df_sales                 # Agrupa os dados por cliente para calcular métricas de RFM
    .groupby("Customer ID")  # Agrrupa todas as compras por Customer ID
    .agg(                    # Aplica agregações específicas para cada métrica RFM
        Recency=("InvoiceDate", lambda x: (snapshot_date - x.max()).days), # Recency: quanto dias desde a última compra do cliente
        Frequency=("Invoice", "nunique"),  #Frequency: número de pedidos únicos realizados pelo cliente
        Monetary=("Revenue", "sum")
    )
)
    

In [11]:
# Normalização

scaler = StandardScaler() # Cria o objeto StandardScaler para normalizar os dados
rfm_scaled = scaler.fit_transform(rfm) # Normaliza colunas FM fit_transform aprende parâmetros e aplica transformação

rfm_scaled = pd.DataFrame(
    rfm_scaled,
    index=rfm.index,
    columns=["Recency", "Frequency", "Monetary"]
)  # Converte array numpy em dataframe mantem indices (Customer ID) e nomes das colunas


In [13]:
# KMeans (k fixo = 4) Segmento de cliente em clusters comportamentais

kmeans = KMeans(n_clusters=4, random_state=42) # Instancia o modelo KMeans com: n_clusters=4 define o número fixo de grupos e random_satate garante reprodutibilidade
rfm["Cluster"] = kmeans.fit_predict(rfm_scaled) # fit_predict treina o modelo e retorna o cluster de cada cliente


In [14]:
# Leitura dos clusters via média

cluster_summary = (
    rfm                      # Agrupa cliente por cluster para interpretar o comportamento médio
    .groupby("Cluster")      # Agrupa pela coluna Cluster
    .mean()                  # Calcula a média de Recency, Frequency e Monetary
    .round(2)                # Arredonda para facilitar leitura humana
)

cluster_summary              # Exibe a tabela resumo dos clusters

,Recency,Frequency,Monetary
Cluster,,,
0,43.70,3.68,1359.06
1,248.08,1.55,480.62
2,7.38,82.54,127338.31
3,15.50,22.33,12709.09


In [16]:
# Contagem por cluster (número de clientes)

rfm["Cluster"].value_counts()

Cluster
0    3054
1    1067
3     204
2      13
Name: count, dtype: int64

## Mineração de Dados - Segmentação RFM

A análise de valor RFM permitiu identificar grupos distintos de clientes com padrões claros de valor, recorrência e engajamento. Esses clusters servem como base estrátégica de retenção, reativação e priorização comercial.

## Simulações de Marketing e vendas

Com base nos clusters RFM devemos priorizar ações reais de negocio em Clientes "VIP"(Cluster 2) com estratégias de retennção prioritárias devido a alta demanda, clientes em crescimento(Cluster 3)  representam potencial de upsell. Clientes de baixo valor ou inativos recomenda-se ações de baixo custo e automação.